БИБЛИОТЕКИ

In [ ]:
# библиотеки....
import numpy as np 
import pandas as pd 
import time
import os
import re
from datetime import datetime

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ФУНКЦИИ ПАРСИНГА....

In [ ]:
# преобразование str в float (в случае пустой строки возврат 0)
def float_zero(arg):
  try:
    res = float(arg)
  except:
    res = 0
  return res

# преобразование str в float (в случае пустой строки возврат None)
def float_null(arg):
  try:
    res = float(arg)
  except:
    res = None
  return res

# преобразование str в int (в случае пустой строки возврат None)
def int_null(arg):
  try:
    res = int(arg)
  except:
    res = None
  return res

# название месяца в строчный номер с нулем
def month_name_to_num(arg):
  months = ['-', 'ЯНВАРЬ', 'ФЕВРАЛЬ', 'МАРТ', 'АПРЕЛЬ', 'МАЙ', 'ИЮНЬ', 
            'ИЮЛЬ', 'АВГУСТ', 'СЕНТЯБРЬ', 'ОКТЯБРЬ', 'НОЯБРЬ', 'ДЕКАБРЬ']
  i = months.index(arg)          
  if i < 10:
    s = '0' + str(i)
  else:
    s = str(i)
  return s

# чтение текстового файла в переменную...
def read_txt_file(path=str):
  f = open(path, 'r')
  fdata = f.readlines()
  f.close
  return fdata

# извлечение названия станции, календарного года и месяца
def get_stname_year_month(fdata):
  s = [None, None, None]
  for line in fdata:
    if re.search('Станция', line):
      break
  s = line.split()
  s.pop(0)
  n = s.index('Год')
  for i in range(n):
    s.pop(1)
  n = s.index('Месяц')  
  for i in range(n-1):
    s.pop(2)
  s = s[:3]
  return s

# извлечение данных страниц 2, 3, 4, 5... 
def get_pages_2_5(fdata, st_year_month:list, page_num:int, time1:str, time2:str):
  res_list = []
  for i in range(len(fdata)):
    if re.search(r"стр\..{1,}" + str(page_num), fdata[i]):
      pos = i
      break
  pos += 7
  d = 0
  for pos in range(pos, pos + 62):
    s = fdata[pos]
    if not s == '\n':
      rab_list = []
      # 00 - дата/время
      day = re.sub(' ', '0', s[2:4])
      if day == '01':
        d += 1
      if d == 1:
        time = time1
      else:
        time = time2      
      rab_list.append(st_year_month[1] + '-' + month_name_to_num(st_year_month[2]) + '-' + day + ' ' + time)
      # 01 - станция
      rab_list.append(st_year_month[0])
      # 02 - температура воздуха
      rab_list.append(float_null(s[5:10]))
      # 03 - температура почвы
      rab_list.append(float_null(s[11:16]))
      # 04 - температура точки росы
      rab_list.append(float_null(s[17:23]))
      # 05 - парциальное давление воды
      rab_list.append(float_null(s[24:29]))
      # 06 - относительная влажность
      rab_list.append(int_null(s[30:34]))
      # 07 - дефицит насыщения
      rab_list.append(float_null(s[35:40]))
      # 08 - атмосферное давление
      rab_list.append(float_null(s[41:48]))
      # 09 - осадки (мм)
      rab_list.append(float_zero(s[123:128]))
      # 10 - направление ветра
      rab_list.append(int_null(s[113:117]))
      # 11 - скорость ветра
      rab_list.append(int_null(s[118:122]))
      # добавление всей строки в рабочую таблицу
      res_list.append(rab_list)
  return res_list

ВЫПОЛНЯЕМ СЧИТЫВАНИЕ ДАННЫХ ИЗ ФАЙЛОВ...

In [ ]:
path = '/content/drive/My Drive/Bases/WIND_DBASE/Sources'
# рекуррентная функция поиска списка файлов txt в каталоге
file_list = []
def search_txt_files(path):
  for files in os.listdir(path):
    if os.path.isfile(path + '/' + files):
      if files[-4:] == '.txt':
        file_list.append(path + '/' + files)
    elif os.path.isdir(path + '/' + files):
      search_txt_files(path + '/' + files)
# вызов рекуррентного поиска всех txt файлов с занесением их в список file_list
search_txt_files(path)

# обработка txt файлов с занесением данных в список списков source_data_list
source_data_list = [] 
for file_name in file_list:
  n = len(source_data_list)
  fdata = read_txt_file(file_name)
  source_data_list.extend(get_pages_2_5(fdata, get_stname_year_month(fdata), 2, '18:00', '21:00'))
  source_data_list.extend(get_pages_2_5(fdata, get_stname_year_month(fdata), 3, '00:00', '03:00'))
  source_data_list.extend(get_pages_2_5(fdata, get_stname_year_month(fdata), 4, '06:00', '09:00'))
  source_data_list.extend(get_pages_2_5(fdata, get_stname_year_month(fdata), 5, '12:00', '15:00'))
  n = len(source_data_list) - n
  print(file_name, n, 'записей')
print('Обработано файлов:', len(file_list))
# всего считано записей....
print('Всего записей:', len(source_data_list))

/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_4.txt 240 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_9.txt 240 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_6.txt 240 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_a.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_7.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_3.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_2.txt 224 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_5.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_1.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_8.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/АХТЫ/s4154770_c.txt 248 записей
/content/drive/My Drive/Bases/WIND_DBASE/Sources/2011/

In [ ]:
# выведем немного как данные парсятся
for i in range(0, 10):
  print(source_data_list[i])

['2011-04-01 18:00', 'Ахты', 6.3, 4.0, 2.9, 7.56, 79, 2.0, 904.0, 0, 260, 1]
['2011-04-02 18:00', 'Ахты', 5.1, 6.0, 3.4, 7.8, 89, 0.98, 909.9, 0, 100, 3]
['2011-04-03 18:00', 'Ахты', 1.7, 4.0, 1.2, 6.65, 96, 0.25, 909.8, 0, 80, 1]
['2011-04-04 18:00', 'Ахты', 2.8, 4.0, 2.0, 7.07, 95, 0.4, 908.6, 0, 0, 0]
['2011-04-05 18:00', 'Ахты', 4.2, 6.0, 3.5, 7.83, 95, 0.41, 901.1, 0, 90, 1]
['2011-04-06 18:00', 'Ахты', 15.1, 9.0, 1.7, 6.9, 40, 10.3, 895.2, 0, 260, 2]
['2011-04-07 18:00', 'Ахты', 5.9, 7.0, 5.4, 8.99, 97, 0.29, 893.2, 0, 74, 3]
['2011-04-08 18:00', 'Ахты', 4.2, 6.0, 4.0, 8.11, 98, 0.13, 896.3, 0, 80, 3]
['2011-04-09 18:00', 'Ахты', 12.2, 8.0, 0.0, 6.1, 43, 8.1, 887.1, 0, 285, 6]
['2011-04-10 18:00', 'Ахты', 7.5, 5.0, 1.1, 6.6, 63, 3.8, 884.7, 0, 280, 3]


СОЗДАЕМ DATAFRAME....

In [ ]:
# определение имен колонок таблицы
columns_names = ['datetime', 'st_name', 't_air', 't_soil', 't_dew', 
                 'P_part_water', 'rel_hum', 'satur_def', 'P_atm', 
                 'precip', 'wind_dir', 'wind_speed']

# создаем и заполняем словарь с данными....
data_dict = {}
for i in range(len(columns_names)):
  column_list = []
  for j in range(len(source_data_list)):
    column_list.append(source_data_list[j][i])
  data_dict[columns_names[i]] = column_list

# создаем dataframe
df = pd.DataFrame(data_dict)
# превращаем колонку datetime в datetime index
df.index = pd.to_datetime(df.datetime)
# Удаляем колонку datetime
df.drop('datetime', axis=1, inplace=True)
# сортируем данные по времени...
df = df.sort_index() 
# смотрим результат....
df

,st_name,t_air,t_soil,t_dew,P_part_water,rel_hum,satur_def,P_atm,precip,wind_dir,wind_speed
datetime,,,,,,,,,,,
2011-01-01 00:00:00,Махачкала,6.6,7.0,5.5,9.01,93,0.73,1019.2,0.0,300,3
2011-01-01 00:00:00,Дербент,7.9,7.0,5.5,9.00,85,1.60,1018.7,0.0,320,2
2011-01-01 00:00:00,Кочубей,5.4,6.0,5.2,8.82,98,0.14,1019.6,0.0,340,3
2011-01-01 00:00:00,Ахты,2.0,2.0,0.9,6.54,93,0.51,897.5,0.0,85,2
2011-01-01 03:00:00,Ахты,1.8,2.0,1.0,6.57,95,0.38,897.4,0.0,70,2
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 18:00:00,Махачкала,4.6,2.0,2.0,7.07,83,1.45,1029.9,0.0,156,3
2020-12-31 21:00:00,Дербент,3.5,2.0,1.9,7.02,89,0.87,1029.4,0.0,165,1
2020-12-31 21:00:00,Ахты,-0.5,-3.0,-8.3,3.27,56,2.62,908.1,0.0,275,2


НЕМНОГО МОЖНО ПОСМОТРЕТЬ НА ДАННЫЕ....

In [ ]:
# количество записей для каждой станции
df_count = df.groupby('st_name').count()
df_count

,t_air,t_soil,t_dew,P_part_water,rel_hum,satur_def,P_atm,precip,wind_dir,wind_speed
st_name,,,,,,,,,,
Ахты,29224,29224,29224,29224,29224,29224,29224,29224,29224,29224
Дербент,29224,29224,29224,29224,29224,29224,29224,29224,29224,29224
Кочубей,29224,29224,29224,29224,29224,29224,29224,29224,29224,29224
Махачкала,29224,29224,29224,29224,29224,29224,29224,29224,29224,29224


In [ ]:
# количество пропусков в данных
df.isna().sum()

st_name         0
t_air           0
t_soil          0
t_dew           0
P_part_water    0
rel_hum         0
satur_def       0
P_atm           0
precip          0
wind_dir        0
wind_speed      0
dtype: int64

In [ ]:
# какие скорости ветра представлены во всей базе
df['wind_speed'].unique()

array([ 3,  2,  1,  4,  0,  6,  5,  8,  7,  9, 13, 10, 11, 12, 14, 15, 18,
       16])

ОСТАВЛЯЕМ ДАННЫЕ ТОЛЬКО ПО МАХАЧКАЛЕ (И УДАЛЯЕМ  СТОЛБЕЦ "Название станции")

In [ ]:
df = df[df['st_name'] == 'Махачкала']
del df['st_name']
df

,t_air,t_soil,t_dew,P_part_water,rel_hum,satur_def,P_atm,precip,wind_dir,wind_speed
datetime,,,,,,,,,,
2011-01-01 00:00:00,6.6,7.0,5.5,9.01,93,0.73,1019.2,0.0,300,3
2011-01-01 03:00:00,6.2,6.0,5.1,8.75,92,0.73,1018.2,0.0,310,4
2011-01-01 06:00:00,5.8,6.0,4.9,8.64,94,0.58,1018.5,0.0,330,3
2011-01-01 09:00:00,5.9,7.0,3.8,8.00,86,1.28,1018.4,0.0,310,6
2011-01-01 12:00:00,5.2,6.0,3.0,7.58,86,1.26,1019.0,0.0,320,6
...,...,...,...,...,...,...,...,...,...,...
2020-12-31 09:00:00,4.3,6.0,1.8,7.01,84,1.33,1026.8,0.0,319,3
2020-12-31 12:00:00,4.7,6.0,2.4,7.29,85,1.29,1026.1,0.0,289,2
2020-12-31 15:00:00,4.4,5.0,2.0,7.09,85,1.25,1025.8,0.0,326,1


In [ ]:
# сохраняем результат датафрейма в CSV файл
df.to_csv('/content/drive/My Drive/Bases/makhachkala_weather_2011_2020_1.csv')